# Predictive modeling - An Example 

In this section we apply machine learning methods to a real world data set, the _adult_ data set. The data set is available on the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) and can be assessed and downloaded [here](https://archive.ics.uci.edu/ml/datasets/Adult).


For the purpose of this tutorial we already downloaded the data set. You may find it in the `data` folder (`./data/adult_data.txt`).

Please note that this tutorial bases on a talk given by [Olivier Grisel](https://github.com/ogrisel) and [Tim Head](https://github.com/betatim) at [EuroScipy 2017](https://www.euroscipy.org/2017/). You can watch their tutorial on YouTube ([Part I](https://www.youtube.com/watch?v=Vs7tdobwj1k&index=3&list=PL55N1lsytpbekFTO5swVmbHPhw093wo0h) and [Part II](https://www.youtube.com/watch?v=0eYOhEF_aK0&list=PL55N1lsytpbekFTO5swVmbHPhw093wo0h&index=2)).


**Import libraries**

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Global setting**

In [ ]:
pd.options.display.max_columns = 200
plt.rcParams["figure.figsize"] = [12,6]

## The machine learning model development pipleline

<img src="./_img/ML_scheme.png" style="height: 500px;">

## Data preparation

### Load the data

In [ ]:
filepath = "./data/adult_data.txt"
names = ("age, workclass, fnlwgt, education, education-num, "
         "marital-status, occupation, relationship, race, sex, "
         "capital-gain, capital-loss, hours-per-week, "
         "native-country, income").split(', ')    
data = pd.read_csv(filepath , names=names)

We take a look at the first rows of the data set by calling the `head()` function.

In [ ]:
data.head()

### Explore the data

A good advice before staring with any type of data analysis: __Know your data!__

Hence, we take a  look at the auxiliary data file `data_names.txt`.

In [ ]:
! cat ./data/adult_names.txt

`fnlwgt` stands for "final weight". This is used by the Census Bureau to create "weighted tallies" of any specified socio-economic characteristics of the population. As we are no Census experts and don't know how to properly use that information, we delete the column.

In [ ]:
data = data.drop('fnlwgt', axis=1)

> __The goal as stated in the auxiliary file is to predict whether a person makes over 50K $ a year.__

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.hist(column='education-num', bins=15);

In [ ]:
data.hist(column='age', bins=15);

In [ ]:
data.hist('hours-per-week', bins=15);

In [ ]:
data.groupby('income')['income'].count().plot.bar(rot=0);

In [ ]:
np.mean(data['income'] == ' >50K')

In [ ]:
low_income = data[data['income'] == ' <=50K']
high_income = data[data['income'] == ' >50K']

bins = np.linspace(10, 90, 20)
plt.hist(low_income['age'].values, bins=bins, alpha=0.5, label='<=50K')
plt.hist(high_income['age'].values, bins=bins, alpha=0.5, label='>50K')
plt.legend(loc='best');

### Feature engineering

Split the data set into `target` and `feature` data sets.

In [ ]:
target = data['income']
print("Target variable: ", target.shape)
features_data = data.drop('income', axis=1)
print("Features: ", features_data.shape)

#### Numerical features

In [ ]:
numeric_features = [column_name for column_name in features_data
                    if features_data[column_name].dtype.kind in ('i', 'f')]
numeric_features

In [ ]:
numeric_data = features_data[numeric_features]
numeric_data.head(5)

#### Categorical features

In [ ]:
categorical_data = features_data.drop(numeric_features, axis=1)
categorical_data.head(5)

In [ ]:
categorical_data.describe()

In [ ]:
categorical_data_encoded = categorical_data.apply(lambda x: pd.factorize(x)[0])
categorical_data_encoded.head(5)

Alternatively we could have used use __one-hot encoding__ for categorical features calling `pd.get_dummies(features_data)`. 


In [ ]:
pd.get_dummies(features_data).head()

**Combine numerical and categorical data**

In [ ]:
features = pd.concat([numeric_data, categorical_data_encoded], axis=1)
features.head()

## Training-Test Split

In [ ]:
X = features.values.astype(np.float32)
y = (target.values == ' >50K').astype(np.int32)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print("Training set: ", X_train.shape)
print("Test set: ", X_test.shape)

## Learning Algorithm - Decision Trees

[__Decision Trees__](https://en.wikipedia.org/wiki/Decision_tree_learning) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.


Some advantages of decision trees are:

* Simple to understand and to interpret (white box model). Trees can be visualized.
* Requires little data preparation.
* The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
* Able to handle both numerical and categorical data. Other techniques are usually specialized in analyzing datasets that have only one type of variable. See algorithms for more information.

The disadvantages of decision trees include:

* Decision-tree learners can create over-complex trees that do not generalize the data well. This is called [overfitting](https://en.wikipedia.org/wiki/Overfitting). 

* Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=8)

**Learning parameters on the train set**

In [ ]:
clf.fit(X_train, y_train)

**Predictions on the test set**

In [ ]:
y_pred = clf.predict(X_test)
y_pred.shape

In [ ]:
y_pred[:10]

In [ ]:
y_test[:10]

## Model evaluation

A [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix), also known as an error matrix, is a special kind of contingency table, with two dimensions ("actual" and "predicted"), and identical sets of "classes" in both dimensions (each combination of dimension and class is a variable in the contingency table).

<img src="./_img/ConfusionMatrix.png" style="height: 400px;">

Source: [Harsha Pulletikurti](http://scaryscientist.blogspot.de/2016/03/confusion-matrix.html)

An informative blog post on performance metrics was recently published by [Andrew Long](https://towardsdatascience.com/data-science-performance-metrics-for-everyone-4d68f4859eef).

> **Challenge:** Compute the accuracy of the model on the test set (the average number of times the model predictions in `y_pred` match the true labels in `y_test`).

In [ ]:
## your code here ...

In [ ]:
# %load ./src/_solutions/accuracy.py

### Alternative performance metrics

Next to __accuary__, __precison__, __sensistivy__ and __specisgty__ there are quite some more performance metrics (for an overview visit  [Wikipedia](https://en.wikipedia.org/wiki/Confusion_matrix)). One performance metric of particular interest is the [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) (ROC) curve.

The ROC curve is created by plotting the __true positive rate (TPR)__ (aka sensitivity) against the __false positive rate (FPR)__ (aka 1 − specificity) at various threshold settings. The ROC curve is generated by plotting the cumulative distribution function (__area under the probability distribution (AUC)__ from $-\infty$ to the discrimination threshold) of the detection probability in the y-axis versus the cumulative distribution function of the false-alarm probability on the x-axis.


In scikit-learn some classifiers offer the `predict_proba` method. This method is a (soft) classifier outputting the probability of the instance being in each of the classes.


In [ ]:
predictions = clf.predict_proba(X_test)

In [ ]:
predictions.shape

In [ ]:
predictions[:5]

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[:, 1]
y_pred_proba[:5]

In order to plot a __ROC curve__ we call the `roc_curve` function provided in the `sklearn.metrics` module. The __area under the curve (AUC)__ is computed by the `roc_auc_score` function.


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
auc = np.round(roc_auc_score(y_test, y_pred_proba),4)
print("ROC AUC: {}".format(auc))

In [ ]:
# compute FPR and TPR
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_pred_proba)

# plot
fix, ax = plt.subplots()
ax.plot(fpr_dt, tpr_dt)
ax.set_ylabel("True Positive Rate (TPR, sensitivity)", size=14)
ax.set_xlabel("False Positive Rate (FPR, 1-specificity)", size=14)
ax.set_title("Receiver Operating Characteristic Curve (AUC {})".format(auc), size=18)
ax.grid();

## Cross-validation


### K-fold cross-validation (Source: [scikit-learn](http://scikit-learn.org/stable/modules/cross_validation.html))

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called __overfitting__. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set `X_test`, `y_test`. 

When evaluating different settings (__*hyperparameters*__) for estimators,there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called __“validation set”__: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called [__cross-validation__][1] (__CV__ for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called __k-fold CV__, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). 

The __performance measure reported by k-fold cross-validation is then the average of the values computed in the loop__. This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.


![](./_img/K-fold_cross_validation_EN.png)

Source: [Wikipedia][1]

[1]: https://en.wikipedia.org/wiki/Cross-validation_(statistics)

In [ ]:
from sklearn.model_selection import cross_validate

clf = DecisionTreeClassifier(max_depth=10)

results = cross_validate(clf, X_train, y_train, cv=5, scoring='roc_auc',
                         return_train_score=True)

In [ ]:
results

In [ ]:
print("ROC AUC Decision Tree (on validation folds): {:.4f} +/-{:.4f}".format(
    np.mean(results['test_score']), 
    np.std(results['test_score'])))

print("ROC AUC Decision Tree (on train folds): {:.4f} +/-{:.4f}".format(
    np.mean(results['train_score']), 
    np.std(results['train_score'])))

> **Challenge**: Try different values of `max_depth` such as: `1`, `2`, `5`, `10`,`15`,... Can you suggest an explanation for the impact of `max_depth` on the cross-validate score?

## Learning Algorithm - Ensemble Learners

[Ensemble learning methods](https://en.wikipedia.org/wiki/Ensemble_learning) use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms alone. The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator. 


### Random Forests

[Random forests](https://en.wikipedia.org/wiki/Random_forest) are an ensemble learning method for classification and regression, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set. Random forests belong to the group of __averaging methods__. The driving principle is to build several estimators independently and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator because its variance is reduced.

In scikit-learn's [`RandomForestClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) the ` n_estimators` controls the number of trees in the forest. The `max_features` parameter controls the size of the random subsets of features to consider when splitting a node. The `max_depth` parameter corresponds to the maximum depth of the tree.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=30, max_features=10,
                             max_depth=10)
clf

In [ ]:
results = cross_validate(clf, X_train, y_train, cv=5, scoring='roc_auc',
                         n_jobs=-1, return_train_score=True)

print("ROC AUC Random Forest (on validation folds): {:.4f} +/-{:.4f}".format(
    np.mean(results['test_score']), 
    np.std(results['test_score'])))

print("ROC AUC Random Forest (on train folds): {:.4f} +/-{:.4f}".format(
    np.mean(results['train_score']), 
    np.std(results['train_score'])))

### Gradient Boosting

[Gradient Tree Boosting](https://en.wikipedia.org/wiki/Gradient_boosting) (GBT) is a generalization of boosting to arbitrary differentiable loss functions. GBT is an accurate and effective off-the-shelf procedure that can be used for both regression and classification problems. 

The advantages of GBT are:

* Natural handling of data of mixed type (= heterogeneous features)
* Predictive power
* Robustness to outliers in output space (via robust loss functions)

In __boosting methods__, base estimators are built sequentially and one tries to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble.

In scikit-learn's [`GradientBoostingClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) implementation, the number of weak learners (i.e. regression trees) is controlled by the parameter `n_estimators`; The size of each tree can be controlled either by setting the tree depth via `max_depth` or by setting the number of leaf nodes via `max_leaf_nodes`. The `learning_rate` is a hyper-parameter in the range `(0.0, 1.0]` that controls overfitting via shrinkage.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(max_leaf_nodes=5, learning_rate=0.1,
                                 n_estimators=100)
clf 

In [ ]:
results = cross_validate(clf, X_train, y_train, cv=5, scoring='roc_auc',
                         n_jobs=-1, return_train_score=True)

print("ROC AUC Gradient Boosted Trees (on validation folds): {:.4f} +/-{:.4f}".format(
    np.mean(results['test_score']), 
    np.std(results['test_score'])))

print("ROC AUC Gradient Boosted Trees (on train folds): {:.4f} +/-{:.4f}".format(
    np.mean(results['train_score']), 
    np.std(results['train_score'])))

## Model comparison via model evaluation metrics

Note that we actually did not evaluate the model performance of the Random Forest Classifier nor the Gradient Boosting Classifier on a test set. Hence, we have to fit the model first using the `fit` method.

In [ ]:
_ = clf.fit(X_train, y_train)

Now we may evaluate the model performance  on the test set. Once again we consider the [ROC AUC metric](https://de.wikipedia.org/wiki/Receiver_Operating_Characteristic).

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = clf.predict_proba(X_test)[:, 1]
print("ROC AUC for Gradient Boosting: {:0.4f}".format(roc_auc_score(y_test, y_pred_proba)))

In [ ]:
# compute FPR and TPR for gradient boosting model
fpr_gbm, tpr_gbm, _ = roc_curve(y_test, y_pred_proba)

# plot
fix, ax = plt.subplots()
ax.plot(fpr_dt, tpr_dt, label="Decision Tree")
ax.plot(fpr_gbm, tpr_gbm, label="gmn")
ax.set_ylabel("True Positive Rate (TPR, sensitivity)", size=14)
ax.set_xlabel("False Positive Rate (FPR, 1-specificity)", size=14)
ax.set_title("Receiver Operating Characteristic Curve)", size=18)
ax.legend()
ax.grid();

In [ ]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=data['income'].unique()))

## Variable importances


In [ ]:
clf.feature_importances_

In [ ]:
ordering = np.argsort(clf.feature_importances_)[::-1]

importances = clf.feature_importances_[ordering]
feature_names = features.columns[ordering]

fig, ax = plt.subplots()
y = np.arange(len(feature_names))
ax.barh(y,importances,align='center')
ax.invert_yaxis() 
ax.set_yticks(y)
ax.set_yticklabels(feature_names, size=14)
ax.set_title("Variable importance", size=18);